# Решение задач многомерной оптимизации

In [ ]:
# Начальная настройка рабочей среды. Запустите эту ячейку перед началом работы!

# Загрузка пакетов
import numpy as np # Работа с массивами
import matplotlib.pyplot as plt # Графики
# Выбор варианта отображения графиков:
# графики будут встраиваться в блокнот:
%matplotlib inline
# графики в отдельном окне:
# %matplotlib
# графики в блокноте, с возможностью масштабирования:
# %matplotlib nbagg
# Более крупный шрифт для графиков по умолчанию
from matplotlib import rcParams
rcParams.update({'font.size': 14})
# Если у вас не видны русские буквы на графиках и нет возможности 
# настротроить matplotlib - раскомментируйте следующую строку.
#rcParams.update({'font.family': 'Arial'})  

import scipy.optimize as so # Пакет с методами оптимизации

import sympy as sp # Пакет символьной математики
sp.init_printing() # Включить отображение выражений sympy в виде математических формул

# Обход проблемы с отображением матриц - определяем функцию для их печати
from IPython.display import  Math
def printMatrix(m):
    """
    Функция для вывода в блокнот матриц SymPy.
    Использование: printMatrix(Матрица)
    """
    return Math(sp.latex(m))

## Задание 1. Оценка параметров эмпирической зависимости

Кинетика связывания $CaO$ изучалась в известняково-пепельных смесях различного состава. Экспериментальные данные, приведенные в таблице, подчиняются уравнению: $$ C = C_0 \cdot e ^{ -\left(\cfrac{kt}{n}\right)^n } $$

Определить [порядок реакции](https://ru.wikipedia.org/wiki/Химическая_кинетика) $n$ и константу скорости $k$ для различных смесей.


Время $t$, мин | Массовая доля $C$, %
:- | :-
0 | 31.5
1 | 15.0
2 | 10.5
4 | 5.2
6 | 2.5
8 | 1.5

Источник: Холоднов В.А. Решение задач безусловной оптимизации с использованием системы компьютерной математики Mathcad. - С.-Пб. :  Санкт-Петербургский государственный технологический институт (технический университет), 2010. - с. 24

### Решение

Задача сводится к выбору таких значений неизвестных параметров $n$ и $k$, которые минимизируют невязку математической модели и экспериментальных данных. Для оценки ошибки модели можно использовать сумму квадратов отклонений предсказанных и фактических концентраций для различных моментов времени $t$:

$$ f(k, n) = \sum\limits_{i=1}^5 (C^{эксп}_i - C^{предск}_i)^2 $$

$C^{эксп}_i$ - указанные в таблице экспериментальные значения концентрации, а $C^{предск}_i$ - рассчитанные по уравнению для момента времени $t$.

Шаг 1. Задаем массивы экспериментальных данных

In [ ]:
# Начальная концентрация
C_0 = 31.5
# Зависимость концентрации от времени реакции
t = np.array([1, 2, 4, 6, 8])
C_exp = np.array([15.0, 10.5, 5.2, 2.5, 1.5])

Шаг 2. Задаем функцию для вычисления концентрации в заданный момент времени

In [ ]:
C_t = lambda t, k, n: C_0 * np.exp(-(k * t / n)**n)

Шаг 3. Задаем функцию для невязки

In [ ]:
@np.vectorize 
def error(k, n):        
    return np.sum((C_exp - C_t(t, k, n))**2)

# @np.vectorize - это специальное указание (декоратор функции): 
# если аргументами функции являются массивы numpy, то функция должна применяться поэлементно
# Такая операция называется _векторизацией_ функции
# Если векторизацию не делать, тогда в формуле для невязки используются массивы 
# и возникает ошибка из-за несовместимости их формы

Шаг 4. Предварительное исследование функции и выбор начального приближения

In [ ]:
# Поскольку в задаче только два параметра, можно оценить область оптимальных значений визуально

k_ = np.linspace(0.25, 0.6, 100)
n_ = np.linspace(0.3, 0.9, 100)

X, Y = np.meshgrid(k_, n_)

Z = error(X, Y)

plt.contourf(X, Y, Z, 20, cmap=plt.cm.viridis)
plt.colorbar()
plt.contour(X, Y, Z, 20, colors='black')
plt.xlabel('k')
plt.ylabel('n')
plt.title('Невязка модели', y=1.02);

Минимум функции достигается вблизи точки $k=0.4$, $m=0.6$. Выберем эту точку в качестве начального приближения

Шаг 5. Оптимизация

In [ ]:
# При оптимизации функций нескольких переменных необходимо, чтобы все переменные находились
# в одном векторном аргументе:

f = lambda X: error(X[0], X[1])

Попытка 1

In [ ]:
# Начальное приближение:
X0 = np.array([0.4, 0.6])

# Минимизация
res = so.minimize(f, X0) # Неудачно!
X_opt = res.x

print(res)
print('\nk* = %.4f, n* = %.4f' %(X_opt[0], X_opt[1]))

Попытка 2 - другое начальное приближение

In [ ]:
# Начальное приближение:
X0 = np.array([0.3, 0.3])

# Минимизация
res = so.minimize(f, X0) 
X_opt = res.x

print(res)
print('\nk* = %.4f, n* = %.4f' %(X_opt[0], X_opt[1]))

Попытка 3 - задаем границы для переменной

In [ ]:
# Начальное приближение:
X0 = np.array([0.4, 0.6])

# Минимизация
res = so.minimize(f, X0, bounds=((0, 1), (0, 1)))
X_opt = res.x

print(res)
print('\nk* = %.4f, n* = %.4f' %(X_opt[0], X_opt[1]))

Попытка 4 - используем безградиентный метод

In [ ]:
# Начальное приближение:
X0 = np.array([0.4, 0.6])

# Минимизация
res = so.minimize(f, X0, method='Nelder-Mead')
X_opt = res.x

print(res)
print('\nk* = %.4f, n* = %.4f' %(X_opt[0], X_opt[1]))

Попытка 5 - используем аналитические выражения для градиента и гессиана

In [ ]:
# Символьное выражение для функции ошибки
n, k = sp.symbols('n, k')

error_s = sp.Rational(0)

for i in range(len(C_exp)):    
    error_s += (C_exp[i] - C_0 * sp.exp(-(k * t[i] / n)**n))**2
    
error_s


In [ ]:
# Градиент функции ошибки
J_error_s = sp.Matrix([error_s]).jacobian([k, n]).T

# Градиент в точке минимума
printMatrix(J_error_s.subs({k:0.41386108,  n:0.67554865}))

In [ ]:
# Получение вычисляемой функции для градиента
J_error = sp.lambdify((k, n), J_error_s, 'numpy')
J_error(0.41386108,  0.67554865)

In [ ]:
# Гессиан функции ошибки - выражение и вычисляемая функция
H_error_s = sp.hessian(error_s, (k, n))
H_error = sp.lambdify((k, n), H_error_s)

In [ ]:
# Гессиан в точке минимума
H_error(0.41386108,  0.67554865)

In [ ]:
# Проверка положительной определенности
np.linalg.eig(H_error(0.41386108,  0.67554865))[0]

In [ ]:
# Функции векторного аргумента для вычисления градиента и гессиана
Jac_error = lambda X: np.array(J_error(X[0], X[1])).ravel()
Hess_error = lambda X: H_error(X[0], X[1])
Jac_error([0.41386108,  0.67554865])

In [ ]:
# Минимизация методом Ньютона

# Начальное приближение:
X0 = np.array([0.4, 0.6])

# Минимизация
res = so.minimize(f, X0, jac=Jac_error, hess=Hess_error, method='Newton-CG')
X_opt = res.x

print(res)
print('\nk* = %.4f, n* = %.4f' %(X_opt[0], X_opt[1]))

In [ ]:
%%timeit
X0 = np.array([0.3, 0.3])

# Минимизация
res = so.minimize(f, X0) 


In [ ]:
%%timeit
# Начальное приближение:
X0 = np.array([0.3, 0.3])

# Минимизация
res = so.minimize(f, X0, jac=Jac_error, hess=Hess_error, method='Newton-CG')


In [ ]:
%%timeit
# Начальное приближение:
X0 = np.array([0.3, 0.3])

# Минимизация
res = so.minimize(f, X0, bounds=((0, 1), (0, 1)))

In [ ]:
%%timeit
# Начальное приближение:
X0 = np.array([0.3, 0.3])

# Минимизация
res = so.minimize(f, X0, method='Nelder-Mead')


### Уменьшение нелинейности целевой функции с помощью логарифмирования

In [ ]:
# Уменьшение нелинейности функции с помощью логарифмирования
ln_f = lambda X: np.log(f(X))

# Начальное приближение:
X0 = np.array([0.3, 0.3])

# Минимизация
res = so.minimize(ln_f, X0, method='CG')
X_opt = res.x

print(res)
print('\nk* = %.4f, n* = %.4f' %(X_opt[0], X_opt[1]))

In [ ]:
# Эффект логарифмирования

k_ = np.linspace(0.25, 0.6, 100)
n_ = np.linspace(0.3, 0.9, 100)

X, Y = np.meshgrid(k_, n_)

Z = error(X, Y)
ln_Z = np.log(Z)

fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(10, 5))

cs0 = ax0.contourf(X, Y, Z, 20, cmap=plt.cm.viridis)
cs1 = ax1.contourf(X, Y, ln_Z, 20, cmap=plt.cm.viridis)
plt.subplots_adjust(wspace=5)
fig.colorbar(cs0, ax=ax0)
fig.colorbar(cs1, ax=ax1)

#plt.colorbar()


ax1.set_xlabel('k')
ax0.set_ylabel('n')
ax0.set_xlabel('k')
ax0.set_title('Невязка модели', y=1.02);
ax1.set_title('Логарифм невязки', y=1.02);

fig.tight_layout()

### Подгонка нелинейной функции с помощью `curve_fit()`

В пакете `scipy.optimize` содержится функция `curve_fit()` специально предназначенная для подгонки параметров кривых к экспериментальным данным на основе критерия наименьших квадратов. Функция использует эффективно работающий с этим типом задач метод оптимизации Левенберга-Марквардта. Помимо оценки параметров функция возвращает также ковариационную матрицу для параметров, позволяющую оценить адекватность модели.

In [ ]:
popt, pcov = so.curve_fit(C_t, t, C_exp, np.array([0.3, 0.3]))
popt

In [ ]:
%%timeit
popt, pcov = so.curve_fit(C_t, t, C_exp, np.array([0.3, 0.3]))

In [ ]:
t_ = np.linspace(0, 10, 101)
plt.plot(t_, C_t(t_, popt[0], popt[1]), label='Модель')
plt.plot(t, C_exp, 'ro', label='Эксперимент')
plt.legend(loc='best')
plt.xlabel('$t$', fontsize=20)
plt.ylabel('$C(t)$', fontsize=20)
plt.title('Кинетика связывания $CaO$', y=1.02);

# Задание 2. Многоступенчатое сжатие газа

При многоступенчатом изоэнтропическом сжатии газа от начального давления $P_0$ до конечного давления $P_N$ желательно установить такие промежуточные давления, при которых суммарная энергия, израсходованная на сжатие, была бы минимальной. Газ охлаждается изобарически до своей начальной температуры после каждого адиабатического сжатия.

Затраты энергии на $n-$й ступени определяются из уравнения:

$$ E_n = \frac{mRT}{a}  \cdot  \left( \left( \frac{p_n} {p_{n-1}} \right)^a -1  \right) $$
    
где 

- $m$ - число кмолей сжимаемого газа, 

- $R$ - универсальная газовая постоянная, $R = 8.314$ кДж / кмоль К 

- $T$ - начальная температура газа, K

- $p_n$ - давление газа после сжатия на $n$-й ступени, атм

- $a$ - константа, зависящая от отношения удельных теплоемкостей газа при постоянном давлении и постоянном объеме $\gamma$:

$$ a = \frac{\gamma - 1}{\gamma} $$
    
Требуется найти такие давления $p_1, p_2..., p_n$, чтобы минимизировать суммарную энергию, затрачиваемую на сжатие:

$$E = \sum\limits_{i=1}^{N} E_i $$

Данные варианта:

![Схема многоступенчатого сжатия](pics/compressors.png "Схема многоступенчатого сжатия")


 - Число ступеней сжатия $N=3$
 
 - Расход газа на входе: $m = 10$ кмоль/с
 
 - Начальная температура газа: $T = 293$ K
 
 - Отношение теплоемкостей при постоянном объеме и постоянном давлении: $\gamma = 1.4$
 
 - Давление газа на входе первой ступени: $p_0 = 1$ атм
 
 - Давление газа на выходе последней ступени: $p_k = 64$ атм


Требуется:

 - Задать функцию для расчета расхода энергии на одной ступени сжатия
 
 - Задать функцию для расчета общего энергопотребления (целевая функция)
 
 - Построить контурный график целевой функции
 
 - С помощью функции `minimize()` пакета `scipy.optimize` найти значения давления на выходе первой и второй ступени, которые минимизируют общее энергопотребление. 
 
 - Повторить расчет для 5 ступеней сжатия. Сравнить общее энергопотребление при использовании 3 и 5 ступеней сжатия (при оптимальных давлениях на промежуточных ступенях).

In [ ]:
# Исходные данные:
gamma = 1.4
a = (gamma - 1) / gamma
p0 = 1
pk = 64
R = 8.314
m = 10
T = 293

## Задание 3. Оптимальная упаковка

Для упаковки продукции требуется спроектировать тару - картонную коробку объемом 50 л. 

Определите оптимальную длину, ширину и высоту коробки, обеспечивающие минимальный расход картона.

Для подбора начальной точки поиска нарисуйте контурный график целевой функции.

Как изменится расход картона, если округлить найденные оптимальные размеры до целого количества см (с сохранением объема)?

Округлить размеры можно с помощью функций `np.floor()` или `np.round()`.

![Схема коробки](pics/box.png "Схема коробки")


Подсказка: эта задача - с ограничением (на объем). Однако можно избавиться от этого ограничения, выразив одну из переменных через объем и две другие переменные.

Источник: Belengundu A., Chandrupatla A. Optimization Concepts and Applications in Engineering. - 2ed. - Prentice-Hall, 2011. - P. 126

## Задание 4. Определение константы скорости и энергии активации по экспериментальным данным

Константа скорости зависит от температуры по закону Аррениуса:

$$ k = k_0 \cdot exp \left(-\frac{E}{RT} \right) $$

По заданной зависимости константы скорости реакции от температуры с помощью метода наименьших квадратов найти наиболее вероятные значения предэкспоненты $k_0$ и энергии активации $E$. 

$ R = 8.314 \frac{Дж}{К\cdot моль} $ - универсальная газовая постоянная



$t,  C^{\circ}$ | $k, с^{-1} $
:- | :-
200 | 10.3
220 | 14.6
240 | 18.1
260 | 25.2
280 | 31.0
300 | 38.8
320 | 50.6

- Построить график изменения концентрации, согласно полученной модели и наложить на него экспериментальные точки

- Построить контурный график функции ошибки (или ее логарифма) в зависимости от параметров

Источник: Холоднов В.А. Решение задач безусловной оптимизации с использованием системы компьютерной математики Mathcad. - С.-Пб. : Санкт-Петербургский государственный технологический институт (технический университет), 2010. - с. 35